<a href="https://colab.research.google.com/github/surabhitri/ResNet20/blob/main/resnet_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training ResNet20 on CIFAR-10


## Import packages 


In [ ]:
# import necessary dependencies
import argparse
import os, sys
import time
import datetime
from tqdm import tqdm_notebook as tqdm
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Setup ResNet20

Setting up the basic block of each layer

In [ ]:
class BasicBlock(nn.Module):

    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=True)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                               stride=1, padding=1, bias=True)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels,
                          kernel_size=1, stride=stride, bias=True),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

Setting up ResNet

In [ ]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3,
                               stride=1, padding=1, bias=True)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)
        self.linear = nn.Linear(64, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, out.size()[3])
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [ ]:
def resnet20():
    return ResNet(BasicBlock, [3, 3, 3])

## Transformations, dataloader, model instantiation, loss function, optimizer

In [ ]:
# useful libraries
import torchvision
import torchvision.transforms as transforms

#############################################
# specify preprocessing function
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010)),
    transforms.RandomCrop(size=[32,32], padding=4),
    transforms.RandomHorizontalFlip(p=0.5)                     
])

transform_val = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010)),
    transforms.RandomCrop(size=[32,32], padding=4),
    transforms.RandomHorizontalFlip(p=0.5)
])
#############################################

# do NOT change these
from tools.dataset import CIFAR10
from torch.utils.data import DataLoader

# a few arguments, do NOT change these
DATA_ROOT = "./data"
TRAIN_BATCH_SIZE = 128
VAL_BATCH_SIZE = 100

#############################################
# construct dataset
train_set = CIFAR10(
    root=DATA_ROOT, 
    mode='train', 
    download=True,
    transform=transform_train    # your code
)
val_set = CIFAR10(
    root=DATA_ROOT, 
    mode='val', 
    download=True,
    transform=transform_val    # your code
)

# construct dataloader
train_loader = DataLoader(
    train_set, 
    batch_size=64,  # your code
    shuffle=True,     # your code
    num_workers=4
)
val_loader = DataLoader(
    val_set, 
    batch_size=64,  # your code
    shuffle=True,     # your code
    num_workers=4
)
#############################################


0it [00:00, ?it/s]

Extracting ./data/cifar10_trainval_F22.zip to ./data
Files already downloaded and verified
Using downloaded and verified file: ./data/cifar10_trainval_F22.zip
Extracting ./data/cifar10_trainval_F22.zip to ./data
Files already downloaded and verified


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:

# specify the device for computation
#############################################

# GPU check                
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device =='cuda':
    print("Run on GPU...")
else:
    print("Run on CPU...")

# model definition
net = resnet20()
net = net.to(device)

#############################################

import torch.nn as nn
import torch.optim as optim

# hyperparameters, do NOT change right now
# initial learning rate
INITIAL_LR = 0.01

# momentum for optimizer
MOMENTUM = 0.9

# L2 regularization strength
REG = 1e-4

#############################################
# create loss function
criterion = nn.CrossEntropyLoss()

# Add optimizer
optimizer = optim.SGD(net.parameters(), lr=INITIAL_LR, momentum=MOMENTUM)
#############################################

Run on GPU...


In [ ]:

# some hyperparameters
# total number of training epochs
EPOCHS = 200

# the folder where the trained model is saved
CHECKPOINT_FOLDER = "./saved_model"

# start the training/validation process
best_val_acc = 0
current_learning_rate = INITIAL_LR

print("==> Training starts!")
print("="*50)
for i in range(0, EPOCHS):
    # handle the learning rate scheduler.
    # if i % DECAY_EPOCHS == 0 and i != 0:
    #     current_learning_rate = current_learning_rate * DECAY
    #     for param_group in optimizer.param_groups:
    #         param_group['lr'] = current_learning_rate
    #     print("Current learning rate has decayed to %f" %current_learning_rate)
    
    #######################
    # switch to train mode
    net.train()
    #######################
    
    print("Epoch %d:" %i)
    # this help you compute the training accuracy
    total_examples = 0
    correct_examples = 0

    train_loss = 0 # track training loss if you want
    
    # Train the model for 1 epoch.
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        ####################################
        # copy inputs to device
        inputs = inputs.to(device)
        
        # compute the output and loss
        outputs = net(inputs)
        #m = nn.Softmax(dim=1)
        #output = m(input)
        train_loss = criterion(outputs, targets.to(device))
        
        # zero the gradient
        optimizer.zero_grad()
        
        # backpropagation
        train_loss.backward()
        
        # apply gradient and update the weights
        optimizer.step()
        
        # count the number of correctly predicted samples in the current batch
        #print((m(outputs))[0])
        #print(predicted)
        #break
        _, predicted = torch.max(outputs, 1)
        total_examples += targets.to(device).size(0)
        correct_examples += (predicted == targets.to(device)).sum().item()
        ####################################
                
    avg_loss = train_loss / len(train_loader)
    avg_acc = correct_examples / total_examples
    print("Training loss: %.4f, Training accuracy: %.4f" %(avg_loss, avg_acc))

    # Validate on the validation dataset
    #######################
    # switch to eval mode
    net.eval()
    
    #######################

    # this help you compute the validation accuracy
    total_examples = 0
    correct_examples = 0
    
    val_loss = 0 # again, track the validation loss if you want

    # disable gradient during validation, which can save GPU memory
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            ####################################
            # copy inputs to device
            inputs = inputs.to(device)
            
            # compute the output and loss
            outputs = net(inputs)
            val_loss = criterion(outputs, targets.to(device))
            
            # count the number of correctly predicted samples in the current batch
            _, predicted = torch.max(outputs, 1)
            total_examples += targets.to(device).size(0)
            correct_examples += (predicted == targets.to(device)).sum().item()
            ####################################

    avg_loss = val_loss / len(val_loader)
    avg_acc = correct_examples / total_examples
    print("Validation loss: %.4f, Validation accuracy: %.4f" % (avg_loss, avg_acc))
    
    # save the model checkpoint
    if avg_acc > best_val_acc:
        best_val_acc = avg_acc
        #if not os.path.exists(CHECKPOINT_FOLDER):
        #    os.makedirs(CHECKPOINT_FOLDER)
        #print("Saving ...")
        #state = {'state_dict': net.state_dict(),
        #         'epoch': i,
        #         'lr': current_learning_rate}
        #torch.save(state, os.path.join(CHECKPOINT_FOLDER, 'simplenn.pth'))
        
    print('')

print("="*50)
print(f"==> Optimization finished! Best validation accuracy: {best_val_acc:.4f}")

==> Training starts!
Epoch 0:
Training loss: 0.0017, Training accuracy: 0.4233
Validation loss: 0.0196, Validation accuracy: 0.3830

Epoch 1:
Training loss: 0.0014, Training accuracy: 0.6127
Validation loss: 0.0031, Validation accuracy: 0.6014

Epoch 2:
Training loss: 0.0017, Training accuracy: 0.6862
Validation loss: 0.0109, Validation accuracy: 0.6884

Epoch 3:
Training loss: 0.0023, Training accuracy: 0.7255
Validation loss: 0.0019, Validation accuracy: 0.7192

Epoch 4:
Training loss: 0.0010, Training accuracy: 0.7560
Validation loss: 0.0019, Validation accuracy: 0.7586

Epoch 5:
Training loss: 0.0008, Training accuracy: 0.7796
Validation loss: 0.0048, Validation accuracy: 0.7484

Epoch 6:
Training loss: 0.0013, Training accuracy: 0.7905
Validation loss: 0.0096, Validation accuracy: 0.7402

Epoch 7:
Training loss: 0.0010, Training accuracy: 0.8046
Validation loss: 0.0084, Validation accuracy: 0.7940

Epoch 8:
Training loss: 0.0015, Training accuracy: 0.8148
Validation loss: 0.0023, 

## Trying different optimizer, learning rate scheduler, and gradient clipping

In [ ]:

# specify the device for computation
#############################################

# GPU check                
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device =='cuda':
    print("Run on GPU...")
else:
    print("Run on CPU...")

# model definition
net = resnet20()
net = net.to(device)

#############################################

import torch.nn as nn
import torch.optim as optim

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

# hyperparameters, do NOT change right now
# initial learning rate
INITIAL_LR = 0.001

# momentum for optimizer
MOMENTUM = 0.9

# L2 regularization strength
weight_decay = 1e-4

# create loss function
criterion = nn.CrossEntropyLoss()

# Add adam optimizer
optimizer = torch.optim.Adam(net.parameters(), INITIAL_LR, weight_decay=weight_decay)

# Set up one-cycle learning rate scheduler
# sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, INITIAL_LR, epochs=8, 
#                                                 steps_per_epoch=len(train_loader))
sched = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[80,120,160,180])
    
#############################################

Run on GPU...


In [ ]:

# some hyperparameters
# total number of training epochs
EPOCHS = 200
grad_clip = 0.1

# the folder where the trained model is saved
CHECKPOINT_FOLDER = "./saved_model"

# start the training/validation process
best_val_acc = 0
current_learning_rate = INITIAL_LR

print("==> Training starts!")
print("="*50)
for i in range(0, EPOCHS):
    # handle the learning rate scheduler.
    # if i % DECAY_EPOCHS == 0 and i != 0:
    #     current_learning_rate = current_learning_rate * DECAY
    #     for param_group in optimizer.param_groups:
    #         param_group['lr'] = current_learning_rate
    #     print("Current learning rate has decayed to %f" %current_learning_rate)
    
    #######################
    # switch to train mode
    net.train()
    #######################
    
    print("Epoch %d:" %i)
    # this help you compute the training accuracy
    total_examples = 0
    correct_examples = 0

    train_loss = 0 # track training loss if you want
    
    lrs = []
    
    # Train the model for 1 epoch.
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        ####################################
        # copy inputs to device
        inputs = inputs.to(device)
        
        # compute the output and loss
        outputs = net(inputs)
        #m = nn.Softmax(dim=1)
        #output = m(input)
        train_loss = criterion(outputs, targets.to(device))
        
        # zero the gradient
        optimizer.zero_grad()
        
        # backpropagation
        train_loss.backward()

        # Gradient clipping
        nn.utils.clip_grad_value_(net.parameters(), grad_clip)
        
        # apply gradient and update the weights
        optimizer.step()

        # Record & update learning rate
        #lrs.append(get_lr(optimizer))
        
        # count the number of correctly predicted samples in the current batch
        #print((m(outputs))[0])
        #print(predicted)
        #break
        _, predicted = torch.max(outputs, 1)
        total_examples += targets.to(device).size(0)
        correct_examples += (predicted == targets.to(device)).sum().item()
        ####################################
                
    avg_loss = train_loss / len(train_loader)
    avg_acc = correct_examples / total_examples
    print("Training loss: %.4f, Training accuracy: %.4f" %(avg_loss, avg_acc))

    # Validate on the validation dataset
    #######################
    # switch to eval mode
    net.eval()
    
    #######################

    # this help you compute the validation accuracy
    total_examples = 0
    correct_examples = 0
    
    val_loss = 0 # again, track the validation loss if you want

    # disable gradient during validation, which can save GPU memory
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            ####################################
            # your code here
            # copy inputs to device
            inputs = inputs.to(device)
            
            # compute the output and loss
            outputs = net(inputs)
            val_loss = criterion(outputs, targets.to(device))
            
            # count the number of correctly predicted samples in the current batch
            _, predicted = torch.max(outputs, 1)
            total_examples += targets.to(device).size(0)
            correct_examples += (predicted == targets.to(device)).sum().item()
            ####################################

    sched.step()
    avg_loss = val_loss / len(val_loader)
    avg_acc = correct_examples / total_examples
    print("Validation loss: %.4f, Validation accuracy: %.4f" % (avg_loss, avg_acc))
    
    # save the model checkpoint
    if avg_acc > best_val_acc:
        best_val_acc = avg_acc
        #if not os.path.exists(CHECKPOINT_FOLDER):
        #    os.makedirs(CHECKPOINT_FOLDER)
        #print("Saving ...")
        #state = {'state_dict': net.state_dict(),
        #         'epoch': i,
        #         'lr': current_learning_rate}
        #torch.save(state, os.path.join(CHECKPOINT_FOLDER, 'simplenn.pth'))
        
    print('')

print("="*50)
print(f"==> Optimization finished! Best validation accuracy: {best_val_acc:.4f}")

torch.save(net, '/content/drive/MyDrive/Deep Learning/Assignment 2/resnet20_lr_manyepochs_adam.pth')

## Making predcition on the test set

In [ ]:
from tools.dataset import CIFAR10
from torch.utils.data import DataLoader

DATA_ROOT = "./data"
BATCH_SIZE = 100

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

test_set = CIFAR10(
    root=DATA_ROOT, 
    mode='test', 
    download=True,
    transform=transform_test
)


test_loader = DataLoader(
    test_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=1)

#########################################################
# use your model to generate predictions on test data
# and save the results into variable "results"
# "results" should be either a numpy array or a torch tensor with length of 10000

# initialize a resnet and load trained weights
net = resnet20()
net = torch.load('/content/drive/MyDrive/Deep Learning/Assignment 2/resnet20_lr_manyepochs_adam.pth') # change the path to your own checkpoint file
#net.load_state_dict(state_dict['state_dict'])
net.cuda()

# remember to switch to eval mode whenever you are making inference
net.eval()

results = []
with torch.no_grad():
    for x in test_loader:
        results.append(
            net(x.cuda()).argmax(1)
        )

# convert results to numpy array
results = torch.cat(results).cpu().numpy()
assert len(results) == 10000

#########################################################
with open('/content/drive/MyDrive/Deep Learning/Assignment 2/predictions_manyepochs.csv', 'w') as fp:
    fp.write("Id,Label\n")
    for i in range(len(results)):
        fp.write("%d,%d\n" %(i, results[i]))

Using downloaded and verified file: ./data/cifar10_test_F22.zip
Extracting ./data/cifar10_test_F22.zip to ./data
Files already downloaded and verified
